In [ ]:
!pip install langchain_community
!pip install transformers
!pip install bitsandbytes
!pip install beautifulsoup4
!pip install langchain
!pip install langchain-community==0.2.1 langchain-core==0.2.1
!pip install sentence-transformers
!pip install faiss-gpu
!pip install accelerate
!pip install langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.7 MB/s eta 0:00:00


In [ ]:
import subprocess
import os
import wiki_scrape as ws
import RAG_chain as rc
import argparse
import transformers

In [ ]:
retrieve_movies = False
retrieve_characters = False
retrieve_series = False
chunk_type = "Recursive"
chunk_size = 800
chunk_overlap = 100
model_id = "Ba2han/Phi-3-Medium-Llamaish"

In [ ]:
def chunk_docs(docs):
    headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2"), ("h3", "Header 3"), ("h4", "Header 4")]
    params = {}

    match chunk_type:
        case "Recursive":
            separators = ["\n\n", "\n", "(?<=\. )", " ", ""]
            params["separators"] = separators
            params["chunk_size"] = chunk_size
            params["chunk_overlap"] = chunk_overlap

            chunks = rc.chunk(docs, headers_to_split_on, chunk_type, **params)

    print("Number of chunks: ", len(chunks))
    print("Number of documents: ", len(docs))

    return chunks

Retrieve movies if needed

In [ ]:
    if retrieve_movies:
        ws.export_movies()
    if retrieve_characters:
        ws.export_characters()
    if retrieve_series:
        ws.export_series()


Read the documents

In [ ]:
print("Importing Documents")
docs = rc.read_docs("./web_pages")

Importing Documents


100%|██████████| 114/114 [00:00<00:00, 8282.96it/s]


Chunk documents

In [ ]:
print("Creating chunks")
chunks = chunk_docs(docs)

Creating chunks
Number of chunks:  1296
Number of documents:  114


Create vector store

In [ ]:
print("Creating a vector store")
embedding_model, vector_index = rc.create_vector_index_and_embedding_model(chunks)
print("Generating retriever")
retriever = vector_index.as_retriever()

Creating a vector store
Generating retriever


Create the LLM pipeline

In [ ]:
print("Downloading LLM model")
model, tokenizer = rc.generate_model(model_id)

AttributeError: module 'RAG_chain' has no attribute 'generate_model'

In [ ]:
print("Creating the langchain pipeline")
llm_pipeline = rc.create_pipeline(model, tokenizer)

Create the chain

In [ ]:
print("Creating chain")
chain = rc.create_qa_RAG_chain(llm_pipeline, retriever)